In [ ]:
%pip install transformers datasets torch scikit-learn streamlit
%pip install tf-keras
%pip install pandas

In [ ]:
'../dataset/depression_emo.json'

In [ ]:
import pandas as pd
import json

Creating dataset for intents classification

In [6]:
# Load the JSON file
with open('../dataset/data_full.json', 'r') as f:
    data = json.load(f)
    
# Create DataFrame from the list of lists in 'oos_val'
df_train_raw = pd.DataFrame(data['train'], columns=['query', 'label'])

# Save the DataFrame to a CSV file

df_train_raw.to_csv('data_full.csv', index=False)
df_train_raw

,query,label
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate
...,...,...
14995,can you explain why my card was declined,card_declined
14996,how come starbucks declined my card when i tri...,card_declined
14997,how come my card was not accepted yesterday,card_declined
14998,find out what happened to make my card get dec...,card_declined


In [7]:
#process data for basic intent
whomadeyou_sample = df_train_raw.loc[df_train_raw['label'] == 'who_made_you'].sample(20, random_state=1)
whatcaniaskyou_sample = df_train_raw.loc[df_train_raw['label'] == 'what_can_i_ask_you'].sample(20, random_state=1)
whatsong_sample = df_train_raw.loc[df_train_raw['label'] == 'what_song'].sample(20, random_state=1)
whatareyourhob_sample = df_train_raw.loc[df_train_raw['label'] == 'what_are_your_hobbies'].sample(20, random_state=1)
doyouhavepets_sample = df_train_raw.loc[df_train_raw['label'] == 'do_you_have_pets'].sample(20, random_state=1)

whatcaniaskyou_sample.reset_index(drop=True, inplace=True)
whomadeyou_sample.reset_index(drop=True, inplace=True)
doyouhavepets_sample.reset_index(drop=True, inplace=True)
whatareyourhob_sample.reset_index(drop=True, inplace=True)
whatsong_sample.reset_index(drop=True, inplace=True)

merged_df = pd.concat([whomadeyou_sample, whatcaniaskyou_sample, whatareyourhob_sample, whatsong_sample, doyouhavepets_sample], ignore_index=True)
merged_df.loc[:,'label'] = 'smalltalk'
df_train_raw = pd.concat([df_train_raw, merged_df], ignore_index=True)
df_train_raw = df_train_raw.sample(frac=1, random_state=42).reset_index(drop=True)
df_train_raw

,query,label
0,i need more checkbooks delivered to me by mail,order_checks
1,id like a car rental between january 1st to th...,car_rental
2,order some new checks to replace the ones i ra...,order_checks
3,make a pto request for dates november 11 to bl...,pto_request
4,what's the purpose of existence,meaning_of_life
...,...,...
15095,how do i speed up the speech of this ai,change_speed
15096,how much gas is in my car,gas
15097,that’s not correct,no
15098,what do i need to make a vacation request with...,pto_request


In [8]:
basic_intents = ['greeting', 'goodbye', 'smalltalk']
df_train = df_train_raw.loc[df_train_raw['label'].isin(basic_intents)].reset_index(drop=True)
df_train.rename(columns={'query': 'text', 'label': 'intent'}, inplace=True)
df_train

,text,intent
0,what types of activities do you enjoy,smalltalk
1,cats or dogs,smalltalk
2,tell me what company programmed you,smalltalk
3,can you tell me about the pets you have,smalltalk
4,see ya later,goodbye
...,...,...
295,what questions do you respond to,smalltalk
296,ai how are you doing today,greeting
297,see you around,goodbye
298,peace out,goodbye


In [12]:
df_suicide = pd.read_csv('../dataset/suicide_detection.csv')
df_suicide = df_suicide.sample(frac=0.5, random_state=42)

df_suicide.to_csv('../dataset/suicide_sampled.csv', index=False)
df_suicide = pd.read_csv('../dataset/suicide_sampled.csv')
df_suicide.head()

,Unnamed: 0,text,class
74414,111734,I Don't know?7? Months self harm free and the ...,suicide
149516,224358,I HAVE TO START BECOMING RICH I HAVE TO START ...,non-suicide
12484,18790,"A poem (haiku) for u/Me-Game-Dev hi, hello hel...",non-suicide
14043,21196,I've honestly got no idea what to do anymore.I...,suicide
30673,46089,Do you ever just cry? Like you just think abou...,non-suicide


In [13]:
df_suicide.drop(columns=['Unnamed: 0'], inplace=True)
df_suicide.rename(columns={'class': 'intent'}, inplace=True)

In [14]:
df_suicide = df_suicide.loc[df_suicide['intent'] == 'suicide', :]
df_suicide = df_suicide.sample(100, random_state=1).reset_index(drop=True)
df_suicide

,text,intent
0,Later skaterI'm killing myself tonight. I alwa...,suicide
1,I can't stand the pain anymoreI am living in a...,suicide
2,May 1stI've been thinking about doing this for...,suicide
3,"I'm all alone now...I've posted before, but my...",suicide
4,i cant do this anymore lolol :)since the age o...,suicide
...,...,...
95,I have a date set nowIt'll likely be Tuesday u...,suicide
96,I want to be a little kid againIt feels like t...,suicide
97,Stuck please helpI go to a pretty good college...,suicide
98,I don't how much longer I can take thisEvery w...,suicide


In [15]:
df_train = pd.concat([df_suicide, df_train], ignore_index=True)
df_train

,text,intent
0,Later skaterI'm killing myself tonight. I alwa...,suicide
1,I can't stand the pain anymoreI am living in a...,suicide
2,May 1stI've been thinking about doing this for...,suicide
3,"I'm all alone now...I've posted before, but my...",suicide
4,i cant do this anymore lolol :)since the age o...,suicide
...,...,...
395,what questions do you respond to,smalltalk
396,ai how are you doing today,greeting
397,see you around,goodbye
398,peace out,goodbye


In [ ]:
# Load the JSON file
with open('../dataset/depression_emo.json', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

# Convert to DataFrame
report_emo = pd.DataFrame(data)

# View the first few rows
report_emo = report_emo[['post', 'feelings']]
report_emo

,post,feelings
0,"I have people close to me willing to help, but...","[loneliness, lack of energy, hopelessness, sad..."
1,I'm tired of it all and it tears me apart that...,"[lack of energy, hopelessness, self hate, sadn..."
2,The emptiness of losing someone you love? The ...,"[emptiness, sadness, hopelessness, worthlessne..."
3,Hi. Just wanted to say I'm glad I found this s...,[loneliness]
4,"Idk if this is the right subreddit for this, s...","[lack of energy, suicide intent, emptiness, ho..."
...,...,...
95,Today’s my 20th bday and no one has said happy...,"[sadness, loneliness, emptiness, hopelessness,..."
96,I am a bit new to relationships and not the br...,"[emptiness, hopelessness, sadness]"
97,I feel like theres like a voice inside that di...,"[self hate, lack of energy, hopelessness, sadn..."
98,My partner is spiraling - I can’t help her. Sh...,"[self hate, hopelessness, anger, sadness, lone..."


In [24]:
report_emo.rename(columns={'post': 'text', 'feelings': 'intent'}, inplace=True)
report_emo.loc[:, 'intent'] = 'report_emotion'
report_emo

,text,intent
0,"I have people close to me willing to help, but...",report_emotion
1,I'm tired of it all and it tears me apart that...,report_emotion
2,The emptiness of losing someone you love? The ...,report_emotion
3,Hi. Just wanted to say I'm glad I found this s...,report_emotion
4,"Idk if this is the right subreddit for this, s...",report_emotion
...,...,...
95,Today’s my 20th bday and no one has said happy...,report_emotion
96,I am a bit new to relationships and not the br...,report_emotion
97,I feel like theres like a voice inside that di...,report_emotion
98,My partner is spiraling - I can’t help her. Sh...,report_emotion


In [22]:
df_train = pd.concat([report_emo, df_train], ignore_index=True)
df_train

,text,intent
0,"I have people close to me willing to help, but...","[loneliness, lack of energy, hopelessness, sad..."
1,I'm tired of it all and it tears me apart that...,"[lack of energy, hopelessness, self hate, sadn..."
2,The emptiness of losing someone you love? The ...,"[emptiness, sadness, hopelessness, worthlessne..."
3,Hi. Just wanted to say I'm glad I found this s...,[loneliness]
4,"Idk if this is the right subreddit for this, s...","[lack of energy, suicide intent, emptiness, ho..."
...,...,...
495,what questions do you respond to,smalltalk
496,ai how are you doing today,greeting
497,see you around,goodbye
498,peace out,goodbye
